In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from causal_nets import causal_net_estimate
from joblib import Parallel, delayed, cpu_count
from savReaderWriter import SavReaderNp

Code to implement the application of neural nets to the NSW dataset.
# Empirical
## NSW Data

In [2]:
def NSW_analyzer(Y,T,X,hidden_layer_sizes,hidden_layer_sizes_t):
    X_train, X_valid, T_train, T_valid, Y_train, Y_valid = train_test_split(
        X, T, Y, test_size=0.2, random_state=42)
    dropout_rates_t=[.2,.2,.2]
    dropout_rates=None
    #dropout_rates_t=None
    tau_pred, mu0_pred, prob_t_pred, psi_0, psi_1, history, history_ps = causal_net_estimate(
        [X_train, T_train, Y_train], [X_valid, T_valid, Y_valid], [X, T, Y], hidden_layer_sizes=hidden_layer_sizes,
        dropout_rates_t=dropout_rates_t, dropout_rates=dropout_rates, 
        batch_size=None,batch_size_t=None, alpha=0., r_par=0., optimizer='Adam', learning_rate=0.0009,  
        hidden_layer_sizes_t=hidden_layer_sizes_t, max_epochs_without_change=30, max_nepochs=10000, seed=42, estimate_ps=True, verbose=True)
    mu0_pred = np.concatenate(mu0_pred)
    tau_pred=np.concatenate(tau_pred)
    prob_t_pred = np.concatenate(prob_t_pred)
    prob_t_pred[prob_t_pred < 0.0001] = 0.0001
    prob_t_pred[prob_t_pred > 0.9999] = 0.9999
    support_frame=pd.DataFrame({'T':T, 'Y':Y, 'mu0_pred':mu0_pred,'tau_pred':tau_pred,'prob_t_pred':prob_t_pred})
    half=support_frame[support_frame['T'] == 1]
    min_support=min(half["prob_t_pred"])
    max_support=max(half["prob_t_pred"])
    support_frame = support_frame.drop(support_frame[support_frame["prob_t_pred"]<min_support].index)
    support_frame = support_frame.drop(support_frame[support_frame["prob_t_pred"]>max_support].index)
    
    p=np.mean(support_frame["prob_t_pred"])
    first_part = (1-support_frame["T"]) * (support_frame["Y"]-support_frame["mu0_pred"])
    second_part = support_frame["T"] * (support_frame["Y"]-support_frame["mu0_pred"]-support_frame["tau_pred"])


    ###define ATT from farrell 2015

    part_one_one= (support_frame["T"]*(support_frame["mu0_pred"]+support_frame["tau_pred"]))/np.mean(support_frame["T"])

    part_one_zero= (support_frame["T"]*(support_frame["mu0_pred"]))/np.mean(support_frame["T"])
    
    part_two_one= ((support_frame["prob_t_pred"])/np.mean(support_frame["T"]))*(support_frame["T"]*((support_frame["Y"]-support_frame["mu0_pred"]-support_frame["tau_pred"])/support_frame["prob_t_pred"]))
    part_two_zero= ((support_frame["prob_t_pred"])/np.mean(support_frame["T"]))*(((1-support_frame["T"])*((support_frame["Y"]-support_frame["mu0_pred"])))/(1-support_frame["prob_t_pred"]))                                                         

    psi_one= part_one_one + part_two_one 
    psi_zero= part_one_zero + part_two_zero 
                                                                                          
    te=np.mean(psi_one-psi_zero)
    se=np.std(psi_one - psi_zero)/np.sqrt(len(psi_one))
    return(te,se)

### 1. Experimental Controls

In [3]:
NSW_1 = pd.read_stata("NSW/nsw_dw.dta")
NSW_1.head()
Y=NSW_1["re78"]
T=NSW_1["treat"]
X=NSW_1.iloc[:,2:10]

In [380]:
att1_1, se1_1=NSW_analyzer(Y,T,X,[20,15,5],[20,10,5])
att1_2, se1_2=NSW_analyzer(Y,T,X,[60,30,20],[20,10,5])
att1_3, se1_3=NSW_analyzer(Y,T,X,[80,80,80],[20,10,5])
att1_4, se1_4=NSW_analyzer(Y,T,X,[20,15,10,5],[20,10,5])
att1_5, se1_5=NSW_analyzer(Y,T,X,[60,30,20,10],[20,10,5])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 20)           180         ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 20)           0           ['dense[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 15)           315         ['dropout[0][0]']                
                                                                                              

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the causal coefficients neural network:
1 epoch - loss: 69548392.0000  val loss: 81807856.0000
26 epoch - loss: 68054712.0000  val loss: 78327408.0000
51 epoch - loss: 64107484.0000  val loss: 70148088.0000
76 epoch - loss: 62549476.0000  val loss: 67569608.0000
101 epoch - loss: 62091080.0000  val loss: 67228880.0000
126 epoch - loss: 61635168.0000  val loss: 67319512.0000
Training is finished.

14/14 [==============================] - 0s 2ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 20)                180       
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                         

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1 epoch - loss: 62.2077  val loss: 12.8453
26 epoch - loss: 43.3580  val loss: 3.5010
51 epoch - loss: 11.1316  val loss: 2.0803
76 epoch - loss: 9.3839  val loss: 1.1886
101 epoch - loss: 6.1703  val loss: 1.0012
126 epoch - loss: 4.3406  val loss: 0.8980
151 epoch - loss: 4.3053  val loss: 0.7474
176 epoch - loss: 4.1298  val loss: 0.6797
201 epoch - loss: 4.3506  val loss: 0.6875
Training is finished.
14/14 [==============================] - 0s 3ms/step
144739.015625
-0.0009765625
0         72.837563
1         48.169762
2         64.141327
3         59.410980
4         63.472042
           ...     
440   -26066.080078
441   -18543.093750
442        1.769874
443    -8122.338379
444   -17862.373047
Length: 444, dtype: float32
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]       

...

1 epoch - loss: 70.8913  val loss: 49.0553
26 epoch - loss: 58.3323  val loss: 28.0170
51 epoch - loss: 42.9147  val loss: 14.0345
76 epoch - loss: 20.3251  val loss: 6.8308
101 epoch - loss: 10.4256  val loss: 2.7598
126 epoch - loss: 26.8426  val loss: 1.3145
151 epoch - loss: 5.2269  val loss: 0.8262
176 epoch - loss: 5.7038  val loss: 0.7577
201 epoch - loss: 5.0894  val loss: 0.6902
226 epoch - loss: 3.2465  val loss: 0.6931
Training is finished.
14/14 [==============================] - 0s 2ms/step
144087.046875
-0.0009765625
0         77.986259
1         49.638145
2         68.013527
3         62.657520
4         69.845398
           ...     
440   -17699.837891
441   -13940.958984
442    16994.837891
443    -2644.371582
444   -13016.760742
Length: 442, dtype: float32
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 i

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the causal coefficients neural network:
1 epoch - loss: 70153328.0000  val loss: 82558160.0000
26 epoch - loss: 66661516.0000  val loss: 79142552.0000
51 epoch - loss: 63522492.0000  val loss: 74199896.0000
76 epoch - loss: 62429508.0000  val loss: 69666424.0000
101 epoch - loss: 61833308.0000  val loss: 69063752.0000
126 epoch - loss: 61338784.0000  val loss: 67770472.0000
151 epoch - loss: 60892184.0000  val loss: 66717592.0000
176 epoch - loss: 60309124.0000  val loss: 65711484.0000
201 epoch - loss: 59356644.0000  val loss: 64227104.0000
226 epoch - loss: 57739712.0000  val loss: 61993052.0000
251 epoch - loss: 54983588.0000  val loss: 58573248.0000
276 epoch - loss: 51097912.0000  val loss: 54828804.0000
301 epoch - loss: 47413288.0000  val loss: 52004272.0000
326 epoch - loss: 45663524.0000  val loss: 50607972.0000
351 epoch - loss: 45117280.0000  val loss: 49077036.0000
376 epoch - loss: 44654536.0000  val loss: 48362040.0000
401 epoch - loss: 44179816.0000  val los

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the propensity score neural network:
1 epoch - loss: 70.8913  val loss: 49.0553
26 epoch - loss: 58.3323  val loss: 28.0170
51 epoch - loss: 42.9147  val loss: 14.0345
76 epoch - loss: 20.3251  val loss: 6.8308
101 epoch - loss: 10.4256  val loss: 2.7598
126 epoch - loss: 26.8426  val loss: 1.3145
151 epoch - loss: 5.2269  val loss: 0.8262
176 epoch - loss: 5.7038  val loss: 0.7577
201 epoch - loss: 5.0894  val loss: 0.6902
226 epoch - loss: 3.2465  val loss: 0.6931
Training is finished.
14/14 [==============================] - 0s 2ms/step
144087.0625
-0.0009765625
0      11137.013672
1       7362.188965
2       9450.555664
3       8760.613281
4       9489.293945
           ...     
440   -26381.962891
441    -9196.520508
442    27548.417969
443     3330.298584
444    -3138.360352
Length: 442, dtype: float32


In [381]:
print(att1_1,se1_1)
print(att1_2,se1_2)
print(att1_3,se1_3)
print(att1_4,se1_4)
print(att1_5,se1_5)

664.890625 875.7344144699382
1264.4840087890625 800.4714688057649
1595.4166259765625 768.4687160782747
1199.5286865234375 858.0063631483279
1569.9820556640625 746.6227183197132


### 2.PSID Controls

In [382]:
NSW_2 = pd.read_stata("NSW/psid_controls.dta")
NSW_2=NSW_2.append(NSW_1)
NSW_2=NSW_2.drop(NSW_2[(NSW_2['data_id']!='PSID') & (NSW_2['treat']==0)].index)
Y=NSW_2["re78"]
T=NSW_2["treat"]
X=NSW_2.iloc[:,2:10]

In [383]:
att2_1, se2_1=NSW_analyzer(Y,T,X,[20,15,5],[20,10,5])
att2_2, se2_2=NSW_analyzer(Y,T,X,[60,30,20],[20,10,5])
att2_3, se2_3=NSW_analyzer(Y,T,X,[80,80,80],[20,10,5])
att2_4, se2_4=NSW_analyzer(Y,T,X,[20,15,10,5],[20,10,5])
att2_5, se2_5=NSW_analyzer(Y,T,X,[60,30,20,10],[20,10,5])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 20)           180         ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 20)           0           ['dense[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 15)           315         ['dropout[0][0]']                
                                                                                              

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the causal coefficients neural network:
1 epoch - loss: 710635904.0000  val loss: 664600192.0000
26 epoch - loss: 595332352.0000  val loss: 548112192.0000
51 epoch - loss: 310900352.0000  val loss: 283978400.0000
76 epoch - loss: 142510320.0000  val loss: 145242224.0000
101 epoch - loss: 109623224.0000  val loss: 136101392.0000
Training is finished.

76/76 [==============================] - 0s 2ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 20)                180       
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the propensity score neural network:
1 epoch - loss: 83.6518  val loss: 8.1462
26 epoch - loss: 35.0919  val loss: 5.5029
Training is finished.
76/76 [==============================] - 0s 2ms/step
787262.875
-3.0517578125e-05
0         -33.989017
1         -28.253307
2         -46.102009
3        -201.773941
4         -16.982578
           ...      
180    175383.234375
181    272104.718750
182    175504.578125
183    220984.390625
184    306545.968750
Length: 2415, dtype: float32
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 60)           540         ['input_1[0][0]']        

...

1 epoch - loss: 1625.5573  val loss: 389.3723
26 epoch - loss: 585.3481  val loss: 9.7186
Training is finished.
76/76 [==============================] - 0s 3ms/step
787262.9375
-0.00048828125
0     -4.029906e+06
1     -4.162184e+06
2     -3.819260e+06
3     -2.615975e+06
4     -4.388345e+06
           ...     
180    2.026125e+05
181    3.120962e+05
182    1.759383e+05
183    2.082203e+05
184    2.673050e+05
Length: 2415, dtype: float32
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 60)           540         ['input_1[0][0]']                
                                                 

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the causal coefficients neural network:
1 epoch - loss: 760895872.0000  val loss: 706307776.0000
26 epoch - loss: 663515072.0000  val loss: 617916224.0000
51 epoch - loss: 580501760.0000  val loss: 535676960.0000
76 epoch - loss: 368564768.0000  val loss: 329675360.0000
101 epoch - loss: 114479520.0000  val loss: 130419472.0000
126 epoch - loss: 113897368.0000  val loss: 128032088.0000
Training is finished.

76/76 [==============================] - 0s 2ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 20)                180       
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                             

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the propensity score neural network:
1 epoch - loss: 1625.5573  val loss: 389.3723
26 epoch - loss: 585.3481  val loss: 9.7186
Training is finished.
76/76 [==============================] - 0s 2ms/step
787263.0
-0.015625
0     -4.456982e+06
1     -4.830634e+06
2     -4.211924e+06
3     -3.015952e+06
4     -5.102099e+06
           ...     
180    1.854822e+05
181    2.760081e+05
182    1.734600e+05
183    2.066946e+05
184    2.708413e+05
Length: 2415, dtype: float32


In [384]:
print(att2_1,se2_1)
print(att2_2,se2_2)
print(att2_3,se2_3)
print(att2_4,se2_4)
print(att2_5,se2_5)

3677.775634765625 737.1709812949512
3593.18310546875 737.7384459927857
3657.72265625 737.818172437936
-32875852.0 10903932.39759428
-38224084.0 10975754.654550781


### 3. CPS controls

In [385]:
NSW_3 = pd.read_stata("NSW/cps_controls.dta")
NSW_3 = NSW_3.append(NSW_1)
###dropping original sample control variables
NSW_3=NSW_3.drop(NSW_3[(NSW_3['data_id']!='CPS1') & (NSW_3['treat']==0)].index)
Y=NSW_3["re78"]
T=NSW_3["treat"]
X=NSW_3.iloc[:,2:10]

In [386]:
att3_1, se3_1=NSW_analyzer(Y,T,X,[20,15,5],[20,10,5])
att3_2, se3_2=NSW_analyzer(Y,T,X,[60,30,20],[20,10,5])
att3_3, se3_3=NSW_analyzer(Y,T,X,[80,80,80],[20,10,5])
att3_4, se3_4=NSW_analyzer(Y,T,X,[20,15,10,5],[20,10,5])
att3_5, se3_5=NSW_analyzer(Y,T,X,[60,30,20,10],[20,10,5])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 20)           180         ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 20)           0           ['dense[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 15)           315         ['dropout[0][0]']                
                                                                                              

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the causal coefficients neural network:
1 epoch - loss: 308795712.0000  val loss: 320238176.0000
26 epoch - loss: 258361264.0000  val loss: 262748640.0000
51 epoch - loss: 131410368.0000  val loss: 133008088.0000
76 epoch - loss: 64115716.0000  val loss: 65034708.0000
101 epoch - loss: 56411276.0000  val loss: 57378284.0000
126 epoch - loss: 55637892.0000  val loss: 56967180.0000
151 epoch - loss: 55527944.0000  val loss: 56887552.0000
176 epoch - loss: 55451824.0000  val loss: 56813544.0000
201 epoch - loss: 55405128.0000  val loss: 56777728.0000
226 epoch - loss: 55374324.0000  val loss: 56754012.0000
251 epoch - loss: 55344744.0000  val loss: 56725140.0000
276 epoch - loss: 55319756.0000  val loss: 56697608.0000
301 epoch - loss: 55297864.0000  val loss: 56677092.0000
326 epoch - loss: 55277988.0000  val loss: 56662500.0000
351 epoch - loss: 55261152.0000  val loss: 56649380.0000
376 epoch - loss: 55246260.0000  val loss: 56636728.0000
401 epoch - loss: 55230364.0000  v

 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 451
Trainable params: 451
Non-trainable params: 0
_________________________________________________________________


C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...


Training of the propensity score neural network:
1 epoch - loss: 1125.3542  val loss: 283.2949
26 epoch - loss: 434.5020  val loss: 2.0987
Training is finished.
498/498 [==============================] - 1s 2ms/step
5188763.5
-0.03125
0      1.631435e+01
1      4.406907e+01
2      2.066145e+01
3      2.032607e+01
4      1.976984e+00
           ...     
180    1.338494e+06
181    2.101658e+06
182    1.240343e+06
183    1.480814e+06
184    1.935749e+06
Length: 15917, dtype: float32
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 60)           540         ['input_1[0][0]']                
    

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the causal coefficients neural network:
1 epoch - loss: 331683872.0000  val loss: 340572928.0000
26 epoch - loss: 287197408.0000  val loss: 297278272.0000
51 epoch - loss: 249265360.0000  val loss: 256391696.0000
76 epoch - loss: 145256656.0000  val loss: 145712160.0000
101 epoch - loss: 58780356.0000  val loss: 60197080.0000
126 epoch - loss: 57181908.0000  val loss: 58716556.0000
151 epoch - loss: 56402008.0000  val loss: 57809916.0000
176 epoch - loss: 55962268.0000  val loss: 57457696.0000
201 epoch - loss: 55792620.0000  val loss: 57371956.0000
226 epoch - loss: 55703592.0000  val loss: 57257516.0000
251 epoch - loss: 55627864.0000  val loss: 57161428.0000
276 epoch - loss: 55552448.0000  val loss: 57054976.0000
301 epoch - loss: 55472900.0000  val loss: 56970452.0000
326 epoch - loss: 55404204.0000  val loss: 56883240.0000
351 epoch - loss: 55337956.0000  val loss: 56798644.0000
376 epoch - loss: 55271680.0000  val loss: 56713124.0000
401 epoch - loss: 55204752.0000 

C:\Users\hugo_\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)



Training of the propensity score neural network:
1 epoch - loss: 1125.3542  val loss: 283.2949
26 epoch - loss: 434.5020  val loss: 2.0987
Training is finished.
498/498 [==============================] - 1s 2ms/step
5188764.0
-0.0625
0      3.079069e+01
1      2.614427e+01
2      2.608661e+01
3      3.126381e+01
4      8.798021e+00
           ...     
180    1.344714e+06
181    2.077417e+06
182    1.173404e+06
183    1.304160e+06
184    1.831812e+06
Length: 15917, dtype: float32


In [387]:
print(att3_1,se3_1)
print(att3_2,se3_2)
print(att3_3,se3_3)
print(att3_4,se3_4)
print(att3_5,se3_5)

-755.9854125976562 738.2211038268383
-1397.52587890625 748.5120546346571
-841.8257446289062 737.3540477227558
79584328.0 8750302.299017446
-7953137.0 7933678.88364973
